In [86]:
from IPython import display
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from matplotlib import pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple

# Tomb added
import random

In [ ]:
#Download Chorales
data_dir = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/chorales')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'midi',
      origin='https://github.com/jamesrobertlloyd/infinite-bach/tree/master/data/chorales/midi',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print(filenames)
print('Number of files:', len(filenames))

In [88]:
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

In [89]:
seqlen = 64

In [90]:
def get_pianoroll(midi, nn_from, nn_thru, seqlen, tempo):
    pianoroll = midi.get_piano_roll(fs=2*tempo/60) # This is the core line which makes this matrix based on 8th note

    # print(f"seqlen!!{seqlen}")
    # print(f"piano_roll.shape[1] a.k.a song length!{pianoroll.shape[1]}")

    if pianoroll.shape[1] < seqlen:
        raise UnsupportedMidiFileException

    pianoroll = pianoroll[nn_from:nn_thru, 0:seqlen] # Pinoroll's value still NOT binary since it has velocity
    binary_pianoroll = np.heaviside(pianoroll, 0) # converting as a binary matrix
    transposed_pianoroll = np.transpose(binary_pianoroll)
    # print(f"Transposed \n {transposed_pianoroll}, {len(transposed_pianoroll)}")

    def add_rest_nodes(pianoroll): # If all the elemets are zero, the rest node says 1,(else 0)
        rests = 1 - np.sum(pianoroll, axis=1)  # axis=1 is row. the elements in the pianoroll matrix are binary so either 1 or 0
        rests = np.expand_dims(rests, 1)

        return np.concatenate([pianoroll, rests], axis=1) # Concatenate the binary pianoroll matrix and the rest node series matrix and return it
    

    def check_pianoroll_dim(pianoroll):
        rows = len(pianoroll)  # This gives the number of rows
        columns = len(pianoroll[0])  # This assumes all rows have the same length

        print("Number of rows:", rows)
        print("Number of columns:", columns)
        print("Total dimesions of pianoroll is", rows*columns)
    
    check_pianoroll_dim(binary_pianoroll)
    # return binary_pianoroll
    return add_rest_nodes(binary_pianoroll)
    # return transposed_pianoroll
    # return add_rest_nodes(transposed_pianoroll)

#### Read_Midi Explanation

Get Major key(keymode=0) or Minor key(keymode=1)<br>
key_number has values ​​from 0 to 11 for major keys and for minor keys,
12~23 is included, <br> so by **dividing it by 12** and converting it to an integer,<br> 

it will be 0 if it is a major key,<br>
it will be 1 if it is a minor key,<br>

and assign it accordingly to keymode.

In [91]:
def read_midi(filename, sop_alto, seqlen):
  midi = pretty_midi.PrettyMIDI(filename)

  if len(midi.key_signature_changes) !=1: # An Exception error is thrown if there is a modulation(key change)
    raise UnsupportedMidiFileException

  key_number = midi.key_signature_changes[0].key_number # explained in the text
  keymode = np.array([int(key_number / 12)])

  
  _, tempo = midi.get_tempo_changes() # _ can be written as tempo_time but won't be used w/in this function
  if len(tempo) != 1: # counting the number of elements in the tempo array. # The Exception error thrown when tempo changes
    raise UnsupportedMidiFileException

  if sop_alto: # The argument is coming in as boolean, True or False
    if len(midi.instruments) < 2: # The exception thrown if there are less than 2 parts
      raise UnsupportedMidiFileException

    # Get pianoRoll binary Matrix for each of Soprano, alto, bass parts
    pr_s = get_pianoroll(midi.instruments[0], 36, 84, seqlen, tempo[0]) # Get pianoroll's arguments (midi, nn_from, nn_thru, seqlen, tempo):
    pr_a = get_pianoroll(midi.instruments[1], 36, 84, seqlen, tempo[0])
    pr_b = get_pianoroll(midi.instruments[2], 36, 84, seqlen, tempo[0])

    # return pr_s, pr_a, keymode
    return pr_s, pr_a, pr_b, keymode

  else:
    # Get a pianoroll which gathered all the parts
    pr = get_pianoroll(midi, 36, 84, seqlen, tempo[0])
    return pr, keymode

#### Make the training data

In [ ]:
# np.set_printoptions(threshold=np.inf) # Show the entire print, esp Matrix

training_data = [] # the len is 496

for each_file in glob.glob(str(data_dir/"**/*.mid*")):
  try:
     pianoroll, keymode = read_midi(each_file, False, seqlen)
     training_data.append(pianoroll)
  except UnsupportedMidiFileException:
     print("Nah")

### --Experimental or Unused code Below-- 

In [93]:
"""Check the dimension of the matrix"""

rows = len(training_data)
columns = len(training_data[0]) if training_data else 0  # Assuming all rows have the same length

print("Shape of the list:", rows, "x", columns, f"shape{rows, columns}")
print("The total dimension of this matrx is:", rows*columns)

Shape of the list: 496 x 48 shape(496, 48)
The total dimension of this matrx is: 23808
